# Attempts to intergrate Grover's Algorithm into the Top-K Scoring System for Quadratic Speedup of Large Language Models.
This is a primitive experiment as aforementioned in the README.md to use Grover's Algorithm for Vector Similarity Search. These series of experiments 
attempt to speed up Gaia's Vector Search process. Though these were mostly for fun, prototype versions of using Grover's Algorithm with Top-K scoring did make it into the first versions of Gaia though as of v1.0.4 for the sake of an decently operational demo to forgo the custom Top-K scoring system.

# Grover's Algorithm Case Study with GreenQuest

Grover's algorithm allows us to find a particular register or in this case vector in an unordered database with $N$ entries in just $O(\sqrt{N})$ steps, compared to the best classical algorithm taking on average $N/2$ steps, thereby providing a __quadratic speedup__.

For large databases (with a large number of entries, $N$), a quadratic speedup can provide a significant advantage. For a database with one million entries, a quantum computer running Grover's algorithm would need about 1000 runs, while a classical computer would need, on average, $500$k runs.

Research has been shown that any optimal quantum solution to an unstructured search problem has a speed limit of $O(\sqrt{N})$ runtime. This research finding matches the performance of Grover's algorithm, thus proving that the algorithm is asymptotically optimal. In fact, Grover's algorithm can be generalized to accelerate any type of search where one can construct a quantum oracle. 

In the context of GreenQuest's top-k scoring method, Grover's algorithm could be used to significantly improve the efficiency of searching and ranking user activities based on sustainability impact. With a vast database of user actions and their sustainability scores, Grover's algorithm could quickly identify the top-scoring actions. This efficiency is crucial for real-time applications where immediate access to the most impactful sustainability actions is necessary. Integrating Grover's algorithm into GreenQuest could theorectically enhance the platform's capability to offer quicker, more efficient services, thus fostering greater user engagement and promoting effective sustainability practices. While this Quantum Scoring System was never used in the demo version, I thought it would be cool to share it in this notebook the testing methods of what could've been. 

This notebook is a primitive test case to use for replication and shouldn't be considered production grade code.

# Replication 
To replicate, we must first install Qiskit (Tests were done on Quantum Hardware (IOQ) ). Bless the people at AWS who let Free Tier members have 1 Hour of Free Circuit Simulator Time.

In [1]:
!pip3 install qiskit qiskit-aer scikit-learn numpy 

^C


DEPRECATION: torchsde 0.2.5 has a non-standard dependency specifier numpy>=1.19.*; python_version >= "3.7". pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of torchsde or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


     ---------------------------------------- 0.0/49.6 kB ? eta -:--:--
     ---------------------------------------- 49.6/49.6 kB 2.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/5.1 MB ? eta -:--:--
   -- ------------------------------------- 0.3/5.1 MB 7.0 MB/s eta 0:00:01
   ------ --------------------------------- 0.9/5.1 MB 9.2 MB/s eta 0:00:01
   ------------ --------------------------- 1.6/5.1 MB 11.2 MB/s eta 0:00:01
   -------------------- ------------------- 2.6/5.1 MB 14.1 MB/s eta 0:00:01
   ------------------------------- -------- 4.0/5.1 MB 17.2 MB/s eta 0:00:01
   ---------------------------------------  5.1/5.1 MB 19.3 MB/s eta 0:00:01
   ---------------------------------------- 5.1/5.1 MB 18.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/115.3 kB ? eta -:--:--
   ---------------------------------------- 115.3/115.3 kB 7.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   -------------------

Import all neccessary libraries as such. This imports qiskit, numpy and sk-learn

In [55]:
from qiskit import Aer, transpile, assemble
from qiskit.algorithms import AmplificationProblem, Grover
from qiskit.circuit.library import Diagonal
import numpy as np
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Now, these functions below just convert Text to Vector Embeddings, Prepare the Quantum Oracle for Grover's Algorithm for Top K and Constructs the QC Circut for Simulation. In addition, it also adjusts vectors to prevent CircuitErrors if the number of Vectors aren't a power of 2.

In [35]:
# Step 1: Convert Text to Vector Embeddings
def convert_texts_to_vectors(texts):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(texts)
    return X.toarray()


# Step 2: Prepare Oracle for Grover's Algorithm for Top-K Scoring
def prepare_oracle_top_k(query_vector, vectors, k):
    similarities = np.dot(vectors, query_vector)
    top_k_indices = np.argsort(similarities)[-k:]
    diag_elements = np.ones(len(vectors))
    diag_elements[top_k_indices] = -1
    oracle = Diagonal(diag_elements)
    return oracle, top_k_indices, similarities

#Step 3: Run Grover's Algorithm for Top-K Scoring. This does use some timing functions so it isn't as efficent.
def run_grovers_algorithm_top_k(query_vector, vectors, texts, k):
    top_k_results = []
    for _ in range(k):
        start_time = time.time()
        oracle, potential_top_k_indices, similarities = prepare_oracle_top_k(query_vector, vectors, k)
        problem = AmplificationProblem(oracle, is_good_state=lambda x: x in potential_top_k_indices)
        grover = Grover()
        backend = Aer.get_backend('qasm_simulator')
        qc = grover.construct_circuit(problem, power=1)
        qc.measure_all()
        tqc = transpile(qc, backend)
        qobj = assemble(tqc)
        result = backend.run(qobj).result()
        measurements = result.get_counts(qc)
        most_frequent = max(measurements, key=measurements.get)
        found_index = int(most_frequent, 2)
        end_time = time.time()
        # Store results including the text and similarity score
        result_info = {
            'index': found_index,
            'vector': vectors[found_index] if found_index < len(vectors) else None,
            'text': texts[found_index] if found_index < len(texts) else None,
            'score': similarities[found_index],
            'time': end_time - start_time
        }
        top_k_results.append(result_info)

    return top_k_results

# Function to adjust the number of vectors to a power of 2
def adjust_vectors_to_power_of_2(vectors):
    next_power_of_2 = 2 ** np.ceil(np.log2(len(vectors)))
    padding_length = int(next_power_of_2 - len(vectors))
    if padding_length > 0:
        padding = np.zeros((padding_length, vectors.shape[1]))
        vectors = np.vstack([vectors, padding])
    return vectors




I've went ahead here and sent up some fake GreenQuest strings just to simulate real life queries. You can add as many as you realistically want. To do comparision, the first string of the list will be the test string to compare to other strings.

I would like to also forewarn people who are running this code that you may get a circuit error depending on the number of vectors. Just ensure that the number of vectors is a power of 2. I've included a function to help default that, just something to note.

In [75]:
greenquest_text_strings = [
    "Tell me about the Center for Sustainability Education (CSE) and its objectives.",  # Query vector
    "The CSE at Dickinson focuses on integrating sustainability into the college's liberal arts curriculum and promoting civic action among students.",
    "Sustainability and social justice are key components of Dickinson's educational approach, as guided by the Center for Sustainability Education.",
    "Dickinson College's CSE has been instrumental in advancing sustainability and environmental awareness in higher education.",
    "How does the CSE's strategic plan address challenges like reduced staffing and financial resources while maintaining its sustainability goals?",
    "The Center for Sustainability Education emphasizes the importance of global interdependence and equity in its sustainability teachings.",
    "CSE's efforts to make antiracism and social justice central to sustainability practices reflect Dickinson College's commitment to inclusivity.",
    "What new goals has the Center for Sustainability Education set for the 2021-2026 period to enhance its educational and civic impact?",
    "Dickinson's CSE offers students opportunities to develop sustainability skills through various co-curricular programs and off-campus studies.",
    "As a leader in sustainability education, how does the Center for Sustainability Education at Dickinson attract and retain high-quality faculty and staff?",
    # False strings
    "Dickinson College has launched a space program to study the sustainability of life on Mars.",
    "The college's new mascot, a giant walking tree, symbolizes its focus on deep forest ecology.",
    "Dickinson's CSE has developed a groundbreaking technology to turn textbooks into renewable energy.",
    "The college plans to relocate to a fully underwater campus to promote marine biology studies.",
    "A recent study at Dickinson claims that listening to classical music increases plant growth.",
    "Dickinson College is famous for its annual underwater basket weaving competition.",
    "The CSE has introduced a program for training squirrels to assist in campus tree planting efforts."
]




Now, this is the fun part. I've defined the code below to define the Top Vectors. The code below should vectorize everything and run Grover's algorithm for Top-K for scoring. Essentially, primitive Quantum Vector Searching using Top-K scoring. 

In [77]:
# Convert texts to vector embeddings and adjust to power of 2
vectors = convert_texts_to_vectors(greenquest_text_strings)
vectors = adjust_vectors_to_power_of_2(vectors)

# Choose a query vector (This uses the first vector in the list)
query_vector = vectors[0]

# Define the number of top vectors to find (K)
K = 3

# Calculate similarity scores for all vectors using Grover's Algorithm
all_results = []
for i in range(1, len(vectors)):
    result = run_grovers_algorithm_top_k(vectors[i], vectors, greenquest_text_strings, 1)
    all_results.append(result[0])  # Append the first result from the Grover's Algorithm run

# Sort all results by similarity score in descending order
sorted_all_results = sorted(all_results, key=lambda x: x['score'], reverse=True)

# Select the top 3 results
top_3_results = sorted_all_results[:3]

# Output the top 3 sorted results
for result in top_3_results:
    print(f"Index: {result['index']}")
    print(f"Text: {result['text']}")
    print(f"Vector: {result['vector']}")
    print(f"Similarity Score: {result['score']:.4f}")
    print(f"Time taken: {result['time']:.4f} seconds\n")

C:\Users\Ty Chermsirivatana\AppData\Local\Temp\ipykernel_1152\3333038729.py:38: DeprecationWarning: Using a qobj for run() is deprecated as of qiskit-aer 0.9.0 and will be removed no sooner than 3 months from that release date. Transpiled circuits should now be passed directly using `backend.run(circuits, **run_options).
  result = backend.run(qobj).result()


Index: 5
Text: The Center for Sustainability Education emphasizes the importance of global interdependence and equity in its sustainability teachings.
Vector: [0.         0.         0.         0.         0.         0.
 0.         0.14490597 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.20375628
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.18878964 0.         0.         0.31042159 0.
 0.         0.         0.31042159 0.         0.         0.
 0.         0.         0.17582495 0.         0.         0.
 0.31042159 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.31042159 0.22145805
 0.         0.         0.         0.         0.31042159 0.
 0.         0.         0.18878964 0.         0.         0.

# Conclusion
Overall, these experiments did produce some interesting finds, I'm still seeing some perfect similarity scores which is interesting. Either my Top-K implementation isn't working or there's something wrong with the similarity scoring? I may want to consult someone more proficent than me. Maybe it's a issue with Vectorization or Implementation of Grover's? One note is that I'm not a expert, just a simple amateur freshman not even aiming for Computer Science. If there is anything irregular or weird in this findings that could help improve my comptency, please write me at chermsit@dickinson.edu or drop a Github Issue!